In [19]:
import wget
import pandas as pd
import pdftotext
from textblob import TextBlob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [20]:
df = pd.read_csv('data/gender-pay-gap-uk-gov/2019.csv.gz')
df.head()


,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted
0,1LIFE MANAGEMENT SOLUTIONS LIMITED,"Ldh House St Ives Business Park,\r\nParsons Gr...",02566586,"93110,\r\n93130,\r\n93290",11.0,-0.5,81.5,94.2,10.0,11.4,...,57.7,44.2,55.8,https://www.1life.co.uk/gender-pay-gap,Mark Braithwaite (Managing Director),250 to 499,1LIFE MANAGEMENT SOLUTIONS LIMITED,False,05/04/2020 00:00:00,10/03/2020 17:03:17
1,1ST CHOICE STAFF RECRUITMENT LIMITED,"1ST CHOICE RECRUITMENT,\r\n8 St. Loyes Street,...",07972006,78109,-2.3,0.0,-114.8,-249.3,1.1,0.4,...,37.1,50.0,50.0,https://www.1stchoice.net/gender-pay-gap-repor...,Gill Knight (MD),250 to 499,1ST CHOICE STAFF RECRUITMENT LIMITED,False,05/04/2020 00:00:00,24/01/2020 09:37:01
2,1ST HOME CARE LTD.,"14b Dickson Street,\r\nElgin Industrial Estate...",SC272838,"86900,\r\n88100",-2.0,0.5,NaN,NaN,0.0,0.0,...,91.0,9.0,91.0,https://reallifeoptions.org/,Ian Hardcastle (Chief Operating Officer),250 to 499,1ST HOME CARE LTD.,False,05/04/2020 00:00:00,12/03/2020 08:21:06
3,23.5 DEGREES LIMITED,"Unit 3 Hedge End Retail Park, Charles Watts Wa...",08014079,56103,10.0,0.0,79.0,35.0,4.0,2.0,...,70.0,31.0,69.0,https://www.23-5degrees.com/gender-pay-gap,Luca Contardo (CFO),500 to 999,23.5 DEGREES LIMITED,False,05/04/2020 00:00:00,11/11/2019 15:33:45
4,A. & B. GLASS COMPANY LIMITED,"Addison Road,\r\nChilton Industrial Estate,\r\...",01543721,43342,19.0,4.0,42.0,45.0,70.0,41.0,...,24.0,90.0,10.0,NaN,PHILIP FARNELL (GROUP HR MANAGER),250 to 499,A. & B. GLASS COMPANY LIMITED,False,05/04/2020 00:00:00,20/05/2019 16:34:42


In [21]:

df = df[['EmployerName', 'CompanyNumber', 'CompanyLinkToGPGInfo', 'DiffMeanHourlyPercent']]
initial_count = df['EmployerName'].count()
df = df.dropna()
df.head(15)


,EmployerName,CompanyNumber,CompanyLinkToGPGInfo,DiffMeanHourlyPercent
0,1LIFE MANAGEMENT SOLUTIONS LIMITED,02566586,https://www.1life.co.uk/gender-pay-gap,11.0
1,1ST CHOICE STAFF RECRUITMENT LIMITED,07972006,https://www.1stchoice.net/gender-pay-gap-repor...,-2.3
2,1ST HOME CARE LTD.,SC272838,https://reallifeoptions.org/,-2.0
3,23.5 DEGREES LIMITED,08014079,https://www.23-5degrees.com/gender-pay-gap,10.0
5,A.B.M. CATERING LIMITED,04168334,http://www.abmcatering.co.uk/wp-content/upload...,21.7
7,A.G. BARR P.L.C.,SC005653,https://www.agbarr.co.uk/responsibility/we-act...,2.3
8,A.J.N. STEELSTOCK LTD.,00689647,https://www.ajnsteelstock.co.uk/downloads-reso...,10.3
9,AB AGRI LIMITED,00193800,https://www.abagri.com/responsibility/our-poli...,9.4
10,AB WORLD FOODS LIMITED,01400901,https://abworldfoods.com/,-3.2
14,ABERDEEN ASSET MANAGEMENT PLC,SC082015,https://www.standardlifeaberdeen.com/__data/as...,29.7


In [22]:
df['tmp_is_pdf']= df.CompanyLinkToGPGInfo.map(lambda x: 'pdf' in str(x))

def categoriesDiffMeanHourlyPercent(x):
    if x < -25:
        result = '>25% in favour of women'
    elif x < -10:
        result = '10-25% in favour of women'
    elif x < 0:
        result = '0-10% in favour of women'
    elif x < 10:
        result = '0-10% in favour of men'
    elif x < 25:
        result = '10-25% in favour of men'
    else:
        result = '>25% in favour of men'
    return result
df['DiffMeanHourlyPercentCategory']  = df['DiffMeanHourlyPercent'].map(categoriesDiffMeanHourlyPercent)
df = df[df['tmp_is_pdf'] == True]
remaining_count = df['EmployerName'].count()
perc_with_pdf = remaining_count/initial_count*100
print("{:.1f}% of entries had pdf files as company link to report".format(perc_with_pdf))

16.5% of entries had pdf files as company link to report


In [23]:
JOHN_LEWIS_URL = df[df['EmployerName'].str.contains("JOHN LEWIS PLC")].CompanyLinkToGPGInfo.values[0]
ABERDEEN_ASSET_MANAGEMENT = df[df['EmployerName'].str.contains("ABERDEEN ASSET MANAGEMENT PLC")].CompanyLinkToGPGInfo.values[0]
# SOUTH_GLOUCESTER_COUNCIL = df[df['EmployerName'].str.contains("South Gloucestershire Council")].CompanyLinkToGPGInfo.values[0]

                
# wget.download(JOHN_LEWIS_URL, 'data/johnlewis.pdf')
# wget.download(ABERDEEN_ASSET_MANAGEMENT, 'data/aberdeen.pdf')
# wget.download(SOUTH_GLOUCESTER_COUNCIL, 'data/south-gloucester-council.pdf')


In [24]:
df['EmployerName'].values

array(['A.B.M. CATERING LIMITED', 'ABERDEEN ASSET MANAGEMENT PLC',
       'ABERDEEN ASSET MANAGERS LIMITED', 'ACENTA STEEL LIMITED',
       "Acorns Children's Hospice Trust", 'AD Astra Academy Trust',
       'ADULT PLACEMENT SERVICES LIMITED', 'AGRIAL FRESH PRODUCE LTD',
       'AIB GROUP (UK) P.L.C.', 'AIRPORT COORDINATION LIMITED',
       'Aldermore Bank PLC', 'ALDWYCK HOUSING GROUP LIMITED',
       'ALLFORD HALL MONAGHAN MORRIS LIMITED',
       'ALLIANCE TRUST SAVINGS LIMITED', 'ALTRO LIMITED',
       'ANGLIAN WATER SERVICES LIMITED', 'ARDENT HIRE SOLUTIONS LIMITED',
       'Ark Schools', 'ARRIVA CYMRU LIMITED',
       'ARRIVA DURHAM COUNTY LIMITED', 'ARRIVA KENT & SURREY LIMITED',
       'ARRIVA KENT THAMESIDE LIMITED', 'ARRIVA LONDON NORTH LIMITED',
       'ARRIVA LONDON SOUTH LIMITED', 'ARRIVA MERSEYSIDE LIMITED',
       'ARRIVA MIDLANDS LIMITED', 'ARRIVA MIDLANDS NORTH LIMITED',
       'ARRIVA NORTHUMBRIA LIMITED', 'ARRIVA PLC',
       'ARRIVA RAIL LONDON LIMITED', 'ARRIVA RAIL 

In [7]:
with open("data/johnlewis.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)
jl = "\n".join(pdf)
print(jl)


        BE
YO U R S E L F.
       A LWAYS
    R EP O RT 2019

M E S S AG E F RO M T R AC E Y K I L L E N ,
PA RT N E R & E X E C U T I V E D I R E C TO R , P E O P L E
Our business is undergoing a once-in-a-lifetime                So for the first time this year, we are publishing the
transformation, as the changes in retail and wider society     Be Yourself. Always Report which combines gender,
accelerate. So it’s more important than ever for us to         ethnicity, age and health and wellbeing. This report focuses
stay true to our principles and our point of difference:       on how we can bring our vision and our values, especially
our Partners.                                                  Be Yourself. Always – to life.
As an employee-owned business, we’ve always put the            In future years, as we grow our knowledge and evidence base,
happiness and wellbeing of our Partners as our first priority. we hope to expand the scope of this report even further.
But in these chang

In [8]:
jl_blob = TextBlob(jl)
jl_blob.sentiment

Sentiment(polarity=0.1310243043234846, subjectivity=0.4323429067896283)

In [9]:

with open("data/aberdeen.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)
aberdeen = "\n".join(pdf)
print(aberdeen)

                                                                                                                                                                                                   Gender report
                                                                                                                                                                                                      December 2019
We are building an inclusive culture where everyone is valued for who they are and what they bring. Balanced
representation of men and women in the workplace is vital to this, and can improve both company profitability
(by 21%1) and the global economy (by $12 trillion2). We are committed to improving gender balance at all levels
in our business from early careers up to and including the Board.
Our 2020 Commitments As at 30 June 2019                                                                                                                                               

In [10]:
aberdeen_blob = TextBlob(aberdeen)
aberdeen_blob.sentiment

Sentiment(polarity=0.04654448900772432, subjectivity=0.35975007427213307)

In [ ]:
import os
all_text_data = {}
forbidden = 0

for idx, row in tqdm(enumerate(df.iterrows())):
    if row[1]['EmployerName'].startswith("AVIS"):
        continue
    report_url = row[1]['CompanyLinkToGPGInfo']
    try:
        tmp_filename = wget.download(report_url, 'data/tmp.pdf')

        with open("data/tmp.pdf", "rb") as f:
            pdf = pdftotext.PDF(f)
        text = "\n".join(pdf)
        print("{}: OK".format(row[1]['EmployerName']))
        all_text_data.update({str(row[1]['CompanyNumber']): text})
        blob = TextBlob(text)
        sentiment = blob.sentiment
        df.loc[row[0],'ReportSentimentPolarity'] = sentiment.polarity
        df.loc[row[0], 'ReportSentimentSubjectivity'] = sentiment.subjectivity
    except Exception as e:
        print("{}: {}".format(row[1]['EmployerName'],e))
    finally:
        try:
            os.remove('data/tmp.pdf')
        except OSError as e:
          # Oh well
          pass

A.B.M. CATERING LIMITED: HTTP Error 403: Forbidden
ABERDEEN ASSET MANAGEMENT PLC: OK
ABERDEEN ASSET MANAGERS LIMITED: OK
ACENTA STEEL LIMITED: HTTP Error 403: Forbidden
Acorns Children's Hospice Trust: OK
AD Astra Academy Trust: OK
ADULT PLACEMENT SERVICES LIMITED: HTTP Error 404: Not Found
AGRIAL FRESH PRODUCE LTD: OK
AIB GROUP (UK) P.L.C.: HTTP Error 404: Not Found
AIRPORT COORDINATION LIMITED: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)>


In [12]:
import pprint
pprint.pprint(all_text_data)

{'00084167': 'Summary of Arriva UK’s\n'
             '2019 gender pay gap\n'
             '   Pay gap mean                                       Pay gap '
             'median                         Bonus pay gap mean\n'
             '   6.0%                                               1.5%\n'
             '                                                                                                                          '
             '-2.9%\n'
             '                                                                                                  '
             '-28%       -12.1%\n'
             '   5.4%                                               5.9%\n'
             '  22.6%                                              26.8%\n'
             '   Bonus pay gap median                                 Males in '
             'receipt of bonus            Females in receipt of bonus\n'
             '                                 4.6%\n'
             '       0.9%      

 '00517211': 'Enjoying life’s\n'
             'everyday moments\n'
             'Annual Report and Accounts 2019\n'
             '\n'
             'Britvic’s commitment to building\n'
             'brands, continuous innovation and\n'
             'sustainability, alongside its renewed\n'
             'focus on creating a culture of winning\n'
             'together, sets us up to be the world’s\n'
             'most dynamic soft drinks company.\n'
             ' Britvic in                            Chief Executive\n'
             ' numbers                               Officer’s statement\n'
             '                2     Chairman’s\n'
             '                                                       7\n'
             '                      statement\n'
             '                                    6\n'
             '\n'
             'Strategic Report                              54   Nomination '
             'Committee Report                    Cautionary note regarding

             'fleet\n'
             '                                                                            '
             '3    Figures include all office and non-manufacturing '
             'emissions.\n'
             'increased to 27% and we additionally supported carbon '
             'sequestration\n'
             '                                                                            '
             '4    Business travel includes our Brazil business unit for the '
             'first time and 2018 data have\n'
             'through an employee tree planting project in Brazil and '
             'Teisseire’s              been restated for comparability.\n'
             'brand partnership with the National Forests Office in France.\n'
             '      Our investment in operational excellence and efficiency '
             'has\n'
             'counterintuitively caused a temporary increase in our '
             'location-based1           2020 goal:\n'
             'manuf

             'and                 • One or more Non-Executive Directors will '
             'attend at least one of\n'
             '  office facility and was able to see the results of the '
             'investments          the company’s existing employee forums.\n'
             '  made at the site as well as being able to spend time with\n'
             '  Kylemore-based staff. The Irish management team provided '
             'a                The Board acknowledged that this is not one of '
             'the recommended\n'
             '                                                                                                                                                          '
             'Financial Statements\n'
             '  detailed review of the local market including the customer '
             'and            approaches set out in the 2018 Code. Instead, by '
             'adopting a range\n'
             '  competitor landscape as well as the strength and p

             'amounting to 3% or more of the voting rights in the issued '
             'ordinary share capital of the company:\n'
             '                                                                                                                             '
             'Number of     Percentage of\n'
             '                                                                                                                        '
             'ordinary shares     voting rights\n'
             'FMR '
             'LLC                                                                                                                    '
             '18,432,163            6.94%\n'
             'Prudential                                                                                                                 '
             '16,549,600            6.23%\n'
             'APG Asset '
             'Management                                                              

...

             'Arriva       -1.8%   1.3%    -5.7%    -829.4% 12.4%   13.3%   '
             '88.2%   11.8%   94.5%   5.5%    95.0%    5.0%     90.6%   9.4%\n'
             'North\n'
             'West Ltd\n'
             '                                                                                              '
             '2019 UK gender pay gap report     2\n'
             '\n'
             'UK gender pay gap reporting data\n'
             'Continued\n'
             '                Pa\n'
             '               m yg\n'
             '                Paea ap\n'
             '                     n\n'
             '               m yg\n'
             '                Boed ap\n'
             '               ga usnian\n'
             '                Bo p p\n'
             '               ga nusm ay\n'
             '                        ea n\n'
             '                %\n'
             '               re ap p\n'
             '                   m m ay\n'
             '

             'Pay Gap Report 2019 | 5\n'
             '\n'
             'EY | Assurance | Tax | Transactions | Advisory\n'
             'About EY\n'
             'EY is a global leader in assurance, tax, transaction and '
             'advisory\n'
             'services. The insights and quality services we deliver help '
             'build trust\n'
             'and confidence in the capital markets and in economies the '
             'world\n'
             'over. We develop outstanding leaders who team to deliver on our\n'
             'promises to all of our stakeholders. In so doing, we play a '
             'critical role\n'
             'in building a better working world for our people, for our '
             'clients and for\n'
             'our communities.\n'
             'EY refers to the global organization, and may refer to one or '
             'more, of\n'
             'the member firms of Ernst & Young Global Limited, each of which '
             'is\n'
             'a

             'reportable\n'
             'entities\n'
             'Arriva       2.9%    0.5%    N/A      N/A     7.1%    0.0%    '
             '91.8%   8.2%    92.5%   7.5%    90.3%    9.7%     95.5%   4.5%\n'
             'Cymru Ltd\n'
             'Arriva     0.4%      3.8%    -11.1%   20.3%   1.6%    4.4%    '
             '89.7%   10.3%   91.1%   8.9%    95.1%    4.9%     93.1%   6.9%\n'
             'Durham\n'
             'County Ltd\n'
             'Arriva Kent 4.9%     1.6%    N/A      N/A     2.2%    0.0%    '
             '85.3%   14.7%   84.5%   15.5%   88.7%    11.3%    89.1%   10.9%\n'
             '& Surrey\n'
             'Ltd\n'
             'Arriva Kent 5.5%     4.1%    54.5%    50.0% 1.6%      2.7%    '
             '85.6%   14.4%   91.7%   8.3%    93.2%    6.8%     91.8%   8.2%\n'
             'Thameside\n'
             'Ltd\n'
             'Arriva       3.0%    6.4%    -23.4% 27.2%     89.8%   88.0% '
             '87.1%     12.9%   91.7%   8.3%    93.8%    6.2%  

             '         • We’ve promoted a culture of inclusion by hosting a '
             'range of events and talks, including a\n'
             '           week-long 2019 Inclusion Summit which was held over '
             'multiple countries and locations.\n'
             '         • We encourage different perspectives through forums '
             'like the Next Generation Board and\n'
             '           our social responsibility programme, Team Studios.\n'
             '         How and who we hire\n'
             '   2     • Recruitment for all of our senior vacancies in 2019 '
             'had a mixed-gender interview panel.\n'
             '           We’ll ensure that’s the case going forward too.\n'
             '         • We aim to have at least one BAME candidate '
             'shortlisted for every senior vacancy. In 2019\n'
             '           we’ve achieved that for two thirds of our vacancies.\n'
             '         • We work in partnership with agenci

             'ongoing        which gives associates               always '
             'linear or uniform. With\n'
             '                                         review of recruitment '
             'materials     experience working in and            this in mind '
             'we have taken a\n'
             '                                         to ensure they do not '
             'discourage    with our international               more '
             'consistent approach to\n'
             '                                         women from applying for '
             'roles,      offices, were women. We              supporting '
             'working parents.\n'
             '                                         and unconscious bias '
             'training       are confident that increased         This '
             'includes a maternity                     Pauline Caldwell\n'
             '                                         for line managers. '
        

In [32]:
fig = px.scatter(df, y='ReportSentimentPolarity', x='ReportSentimentSubjectivity', color='DiffMeanHourlyPercent', 
                 hover_data=['EmployerName'])
fig.update_layout(title="Companies' internal reports subjectivity vs polarity",
                  yaxis_zeroline=False, xaxis_zeroline=False)
fig.update_xaxes(title_text='Subjectivity')
fig.update_yaxes(title_text='Polarity (less to more positive sentiment)')
fig.show()


In [ ]:
df